In [538]:
import pandas as pd
import numpy as np

In [539]:
from sklearn import datasets
X_data, y_data = datasets.fetch_openml('boston', return_X_y=True)

/Users/kwon-yong-geun/miniconda3/envs/ml/lib/python3.8/site-packages/sklearn/datasets/_openml.py:292: UserWarning: Multiple active versions of the dataset matching the name boston exist. Versions may be fundamentally different, returning version 1.
  warn(
/Users/kwon-yong-geun/miniconda3/envs/ml/lib/python3.8/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [540]:
from sklearn import preprocessing

minmax_scale = preprocessing.MinMaxScaler().fit(X_data)
# standard_scale = preprocessing.StandardScaler().fit(X_data)

x_scaled_data = minmax_scale.transform(X_data)

x_scaled_data[:3]

array([[0.00000000e+00, 1.80000000e-01, 6.78152493e-02, 0.00000000e+00,
        3.14814815e-01, 5.77505269e-01, 6.41606591e-01, 2.69203139e-01,
        0.00000000e+00, 2.08015267e-01, 2.87234043e-01, 1.00000000e+00,
        8.96799117e-02],
       [2.35922539e-04, 0.00000000e+00, 2.42302053e-01, 0.00000000e+00,
        1.72839506e-01, 5.47997701e-01, 7.82698249e-01, 3.48961980e-01,
        4.34782609e-02, 1.04961832e-01, 5.53191489e-01, 1.00000000e+00,
        2.04470199e-01],
       [2.35697744e-04, 0.00000000e+00, 2.42302053e-01, 0.00000000e+00,
        1.72839506e-01, 6.94385898e-01, 5.99382080e-01, 3.48961980e-01,
        4.34782609e-02, 1.04961832e-01, 5.53191489e-01, 9.89737254e-01,
        6.34657837e-02]])

In [541]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_scaled_data, y_data, test_size=0.33)

In [542]:
from sklearn import  linear_model

regr = linear_model.LinearRegression(fit_intercept=True,  
                                    copy_X=True, # X의 값이 바뀔 수 있기 때문에 X의 값을 복사해놓고 학습하겠다는 의미
                                    n_jobs=8) # 몇 개의 cpu를 쓸 것인지
regr.fit(X_train, y_train)

print('Coefficients: ', regr.coef_)
print('intercept: ', regr.intercept_)

Coefficients:  [-12.96737126   3.83461368   1.5241801    4.52659125  -9.22444293
  19.82937951   0.71427628 -14.41565621   7.13852368  -5.33237938
  -9.5115654    2.97539012 -20.85980941]
intercept:  26.75210351291684


In [510]:
import numpy as np
y_data = np.array(y_data).reshape(506, 1)

In [208]:
df = pd.read_csv("./train.csv")
df.head()

,x,y
0,24,21.549452
1,50,47.464463
2,15,17.218656
3,38,36.586398
4,87,87.288984


In [270]:
X = df["x"].values.reshape(-1,1)
y = df["y"].values

In [274]:
ones = np.ones(X.size).reshape(-1, 1)

In [275]:
X = np.concatenate((ones, X), axis=1)

In [281]:
X.shape

(700, 2)

In [282]:
y.shape

(700,)

In [543]:
class LinearRegressionGD(object):
    def __init__(self, fit_intercept=True, copy_X=True,
                            eta0=0.001, epochs=1000, weight_decay=0.9):
        self.fit_intercept = fit_intercept
        self.copy_X = copy_X
        self._eta0 = eta0
        self._epochs = epochs

        self._cost_history = []

        self.theta = None
        self._coef = None
        self._intercept = None
        self._new_X = None
        self._w_history = []
        self._weight_decay = weight_decay

    def cost(self, h, y):
        return (1/(2*len(y))) * np.sum((h-y)**2)

    def hypothesis_function(self, X, theta):
        return X.dot(theta)

    def gradient(self, X, y, theta):
        m = len(y)
        predictions = self.hypothesis_function(X, theta)
        gradient = []

        for i in range(theta.size):
            partial_marginal = X[:, i]
            errors_xi = (predictions - y) * partial_marginal
            gradient.append((1 / m) * errors_xi.sum())
        return np.array(gradient)

    def fit(self, X, y):
        self._new_X = X
        
        if self.fit_intercept == True:
            ones = np.ones(len(self._new_X)).reshape(-1, 1)
            self._new_X = np.concatenate((ones, self._new_X), axis=1)
        
        theta = np.ones(self._new_X.shape[1])

        for epoch in range(self._epochs):

            gradient = self.gradient(self._new_X, y, theta).flatten()

            for i in range(theta.size):
                theta = theta - self._eta0 * gradient

            if epoch % 100 == 0:
                self._w_history.append(theta)
                cost = self.cost(
                    self.hypothesis_function(self._new_X, theta), y)
                self._cost_history.append(cost)
            self._eta0 = self._eta0 * self._weight_decay

        self.theta = theta
        self.coef = theta[1:]
        self.intercept = theta[0]

        return self

    # 마지막
    def predict(self, X):
        if self.fit_intercept == True:
            ones = np.ones(len(X)).reshape(-1, 1)
            X = np.concatenate((ones, X), axis=1)
    
        return X.dot(self.theta)

    @property
    def coef(self):
        return self._coef
    
    @coef.setter
    def coef(self, value):
        self._coef = value

    @property
    def intercept(self):
        return self._intercept
    
    @intercept.setter
    def intercept(self, value):
        self._intercept = value

    @property
    def weights_history(self):
        return np.array(self._w_history)

    @property
    def cost_history(self):
        return self._cost_history

In [544]:
lr_gd = LinearRegressionGD(fit_intercept=False)

In [545]:
lr_gd.fit(X_train, y_train)

In [546]:
lr_gd._coef, lr_gd._intercept

(array([1.32148327, 1.60189569, 1.21371267, 1.53330102, 2.14673479,
        2.17976497, 1.54721104, 1.49349507, 1.59756313, 2.06842927,
        2.86024511, 1.38825773]),
 1.0246018843262803)

In [547]:
lr_gd.predict(X_train).shape

(339,)

In [415]:
theta = np.random.normal((2, 1))

In [416]:
lr_gd.gradient(X, y, theta)

array([  7.70182774, 452.85019811])

In [417]:
lr_gd.fit(X, y)

[0.983723   0.983723   0.96997259]


In [418]:
from sklearn.linear_model import LinearRegression

In [425]:
regr = LinearRegression(fit_intercept= True, copy_X=True, n_jobs=1)
print(X.shape)
regr.fit(X, y)

(700, 2)


LinearRegression(n_jobs=1)

In [426]:
regr.coef_, regr.intercept_

(array([0.        , 1.00077825]), -0.12015553181322502)